## Installation

In [1]:
! pip install wyn-voice ag2 pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.7/328.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


## Imports

In [2]:
from wyn_voice.chat import *
from autogen import ConversableAgent
from pydub import AudioSegment

In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [4]:
from pathlib import Path
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)
executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="NEVER",  # Always take human input for this agent for safety.
)

## Configure and Run

In [42]:
# @title Choose a bot

# Reinitiate bot
# Example usage
assistant_v1 = ChatBot(
    api_key=OPENAI_API_KEY,
    protocol="""
    You are a helpful assistant.

    If user asks you python related question, you will ask question back and clarify it.
    If all makes sense, you will write the python code for user.
    If the python code is written from before, ask user whether they want to execute it.
    """
)
assistant_v2 = ChatBot(
    api_key=OPENAI_API_KEY,
    protocol="""
    You are a helpful assistant. You are an expert to recommend life style choices.
    """
)
assistant_v3 = ChatBot(
    api_key=OPENAI_API_KEY,
    protocol="""
    You are a therapist.

    You are an expert at guiding people through mental health stresses and work situations.
    You are a good listener and you take the patients (user) complain and try to understand it.
    When user ask, you may suggest some life style choices to the user based on the user's complain.
    """
)

# Choose assistant
value = "Coding Helper" # @param ["Coding Helper", "Lifestyle Helper", "Therapist"]

def set_assistant():
    if value == "Coding Helper":
        assistant = assistant_v1
    elif value == "Lifestyle Helper":
        assistant = assistant_v2
    else:
        assistant = assistant_v3
    audio_processor = AudioProcessor(bot=assistant)

    return assistant, audio_processor

assistant, audio_processor = set_assistant()

In [ ]:
# @title Talk to JARVIS

prompt = ""
response = ""
reply = ""
audio_length = 0
while "exit" not in prompt.lower():
    # User
    prompt = audio_processor.voice_to_text(sec=10)
    print("🤔 User: ", prompt)

    # Command [conditional]
    if "execute" in prompt.lower():
        message_with_code_block = f"""This is a message with code block.
        {response}
        """

        # Generate a reply for the given code.
        reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
        print("💻 Executing....")
        print(reply)
        reply = "Execution successful!" if len(reply) > 100 else reply

        # Bot
        try:
            response = assistant.generate_response(
                f"""User has provided the prompt: {prompt}
                And previous we have an answer: {response}

                Now that the code finished executing. Here's the result: {reply}.

                Summarize the result and ask for another question or task. """
            )
            print("🤖 Bot: ", response)
        except:
            assistant, audio_processor = set_assistant()
            response = assistant.generate_response(
                f"""User has provided the prompt: {prompt}
                And previous we have an answer: {response}

                Now that the code finished executing. Here's the result: {reply}.

                Summarize the result and ask for another question or task. """
            )
            print("🤖 Bot: ", response)

    else:
        # Bot
        try:
            response = assistant.generate_response(prompt)
            print("🤖 Bot: ", response)
        except:
            assistant, audio_processor = set_assistant()
            response = assistant.generate_response(prompt)
            print("🤖 Bot: ", response)

    # Audio
    if "\n\n```python" in response.lower() or len(response) > 500:
        response_short = assistant.generate_response(f"Summarize the following in 1-2 sentences: {response}")
    else:
        response_short = response

    # Audio output
    output_file_path = audio_processor.text_to_voice(response_short)
    audio = AudioSegment.from_file(output_file_path)
    audio_length = len(audio) / 1000.0 + 1
    print("⏳ Audio lag: ", audio_length)

    # Wait from last round
    time.sleep(audio_length)